In [22]:
%reload_ext autoreload
%autoreload 2

import papermill as pm

In [23]:
notebooks = [
  '01-data-collection.ipynb',
  '02-exploratory-data-analysis.ipynb',
  '03-data-preparation.ipynb',
  '04-modelisation.ipynb'
]
for notebook in notebooks:
    notebook_path = f"../notebooks/{notebook}"
    output_notebook = f"{'../notebooks/outputs'}/{notebook.replace('.ipynb', '_output.ipynb')}"
    pm.execute_notebook(notebook_path, output_notebook)

Executing:   0%|          | 0/34 [00:00<?, ?cell/s]

Executing:   0%|          | 0/44 [00:00<?, ?cell/s]

Executing:   0%|          | 0/47 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]